#### Imports

In [1]:
import os
import re
import skrf as rf
import pandas as pd

#### Paths

In [2]:
script_directory = os.getcwd()#obtem diretório atual
s2ps_path = os.path.join(script_directory, 'oil_data','oil_data') #cria string caminho para todos os .s2p

#### Regex = hell

In [3]:
regex = r'SAW_06_02_2024_OIL_CANOLA_(\d+)\.S2P'

In [4]:
def read_s2p_vegetal_data(regex, path):
    dfs = []#lista para ocncatenar todos os dicts de cada medição 
    for filename in os.listdir(path):
        match = re.match(regex, filename)
        if match:
            #obtem informação do nome do arquivo
            oil_name = "CANOLA"
            measurement_number = int(match.group(1))
            
            #carrega arquivo
            path_to_oil = os.path.join(path, filename)
            s2p_file = rf.Network(path_to_oil)
            
            #extrai frequência e parâmetros S do objeto Network
            frequency = s2p_file.f
            s_parameters = s2p_file.s_db
            
            #Dataframe com informações do objeto Network
            individual_oil_dict = {
                'OIL NAME': oil_name,
                'MEASUREMENT NUMBER': measurement_number,
                'FREQUENCY (MHZ)':frequency/1000000,
                'S11 (db)' : s_parameters[:,0,0],
                'S21 (db)' : s_parameters[:,1,0],
                'S12 (db)' : s_parameters[:,0,1],
                'S22 (db)' : s_parameters[:,1,1],
            }
            df = pd.DataFrame.from_dict(individual_oil_dict)
            dfs.append(df)
            
    dataframe = pd.concat(dfs, ignore_index=True)
    return dataframe

#### Read Dataframes

In [5]:
df_vegetal = read_s2p_vegetal_data(regex, s2ps_path)

In [6]:
df_vegetal.info

<bound method DataFrame.info of       OIL NAME  MEASUREMENT NUMBER  FREQUENCY (MHZ)  S11 (db)  S21 (db)  \
0       CANOLA                   1       110.000000 -0.939643 -74.32262   
1       CANOLA                   1       110.002000 -0.939678 -83.86302   
2       CANOLA                   1       110.004000 -0.939612 -75.45787   
3       CANOLA                   1       110.006001 -0.938919 -79.45680   
4       CANOLA                   1       110.008001 -0.938572 -79.10075   
...        ...                 ...              ...       ...       ...   
29995   CANOLA                   3       129.991999 -1.224063 -69.69292   
29996   CANOLA                   3       129.993999 -1.224219 -70.35383   
29997   CANOLA                   3       129.996000 -1.223880 -70.20517   
29998   CANOLA                   3       129.998000 -1.225183 -68.79918   
29999   CANOLA                   3       130.000000 -1.225674 -67.78519   

       S12 (db)  S22 (db)  
0     -77.85377 -1.185256  
1     -80.7

In [7]:
#assegura somente óleo de girassol lido
for name in df_vegetal["OIL NAME"]:
    if name != "CANOLA":
        print("Diferente de canola")

In [8]:
#cria .csv caso precise no futuro.
path_save_csv = os.path.join(script_directory, 'oil_data','oil_data','CSVs')
if not os.path.exists(path_save_csv):
   os.makedirs(path_save_csv)
df_vegetal.to_csv(path_save_csv + '/canola_measurements.csv')

In [9]:
def average_measurements(dataframe):
    #agrupa valores por frequência e calcula média do que interessa
    return dataframe.groupby('FREQUENCY (MHZ)', as_index=False)['S21 (db)'].mean()

In [10]:
df_canola_s21 = average_measurements(df_vegetal)

In [11]:
df_canola_s21.info


<bound method DataFrame.info of       FREQUENCY (MHZ)   S21 (db)
0          110.000000 -77.238280
1          110.002000 -81.705140
2          110.004000 -77.912783
3          110.006001 -77.498673
4          110.008001 -79.490893
...               ...        ...
9995       129.991999 -69.731563
9996       129.993999 -69.206113
9997       129.996000 -69.564280
9998       129.998000 -68.434813
9999       130.000000 -67.619743

[10000 rows x 2 columns]>

In [12]:
#cria .csv caso precise no futuro.
path_save_csv_processed = os.path.join(script_directory, 'oil_data','oil_data','CSVs','processed')
if not os.path.exists(path_save_csv_processed):
   os.makedirs(path_save_csv_processed)
df_vegetal.to_csv(path_save_csv_processed + '/canola_measurements_processed.csv')